In [2]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
from dash import dash_table
import datetime
import psycopg2

conn = psycopg2.connect(
    dbname="bhchggi2oszg8ltxos7t",
    user="uo6eisf85mmsla5uznkh",
    password="lk51F5Rf88DRFNcvMQ9up7U34F6kex",
    host="bhchggi2oszg8ltxos7t-postgresql.services.clever-cloud.com",
    port="50013"
)

cur = conn.cursor()

query = """
SELECT * FROM knowledge_base;
"""

# Загрузка данных в DataFrame
df = pd.read_sql_query(query, conn)

metrics_df = pd.read_excel('metrics_in_time.xlsx')

coolors = ['#4da2f1', '#ff3d64', '#8ad554', '#ffc636', '#ff66b2']
text_color = coolors[0]  # Используем первый цвет для всего текста в карточках

app = dash.Dash(__name__, 
               external_stylesheets=[dbc.themes.BOOTSTRAP],
               suppress_callback_exceptions=True)

app.layout = html.Div([
    html.H2('Анализ работы ИИ чат-бота', style={'padding': '20px', 'margin-bottom': '0px'}),
    
    dcc.Tabs(id="main-tabs", value='tab-1', children=[
        dcc.Tab(label='Общая информация', value='tab-1', style={
            'padding': '5px 15px',
            'height': '30px',
            'min-height': '30px',
            'background': 'white',
            'border': '0px solid #d6d6d6',
            'border-bottom': 'none'
        }, selected_style={
            'padding': '5px 15px',
            'height': '30px',
            'min-height': '30px',
            'background': 'white',
            'border': '0px solid #d6d6d6',
            'border-bottom': 'none',
            'font-weight': 'bold'
        }),
        dcc.Tab(label='Вопросы', value='tab-2', style={
            'padding': '5px 15px',
            'height': '30px',
            'min-height': '30px',
            'background': 'white',
            'border': '0px solid #d6d6d6',
            'border-bottom': 'none'
        }, selected_style={
            'padding': '5px 15px',
            'height': '30px',
            'min-height': '30px',
            'background': 'white',
            'border': '0px solid #d6d6d6',
            'border-bottom': 'none',
            'font-weight': 'bold'
        }),
        dcc.Tab(label='Работа модели', value='tab-3', style={
            'padding': '5px 15px',
            'height': '30px',
            'min-height': '30px',
            'background': 'white',
            'border': '0px solid #d6d6d6',
            'border-bottom': 'none'
        }, selected_style={
            'padding': '5px 15px',
            'height': '30px',
            'min-height': '30px',
            'background': 'white',
            'border': '0px solid #d6d6d6',
            'border-bottom': 'none',
            'font-weight': 'bold'
        }),
        dcc.Tab(label='Выгрузка данных', value='tab-4', style={
            'padding': '5px 15px',
            'height': '30px',
            'min-height': '30px',
            'background': 'white',
            'border': '0px solid #d6d6d6',
            'border-bottom': 'none'
        }, selected_style={
            'padding': '5px 15px',
            'height': '30px',
            'min-height': '30px',
            'background': 'white',
            'border': '0px solid #d6d6d6',
            'border-bottom': 'none',
            'font-weight': 'bold'
        }),
    ], style={
        'margin-bottom': '0px',
        'height': '40px',
        'align-items': 'center'
    }),
    
    # Фильтры
    html.Div(id='main-filters-container', children=[
        dbc.Row([
            dbc.Col([
                dcc.Dropdown(
                    id='main-campus-filter',
                    options=[{'label': 'Все кампусы', 'value': 'all'}] + 
                            [{'label': i, 'value': i} for i in df['campus'].unique()],
                    value=['all'],
                    multi=True,
                    placeholder='Выберите кампус(ы)'
                )
            ], width=4),
            
            dbc.Col([
                dcc.Dropdown(
                    id='main-education-filter',
                    options=[{'label': 'Все уровни', 'value': 'all'}] + 
                            [{'label': i, 'value': i} for i in df['education_level'].unique()],
                    value=['all'],
                    multi=True,
                    placeholder='Выберите уровень(и) образования'
                )
            ], width=4),
            
            dbc.Col([
                dcc.Dropdown(
                    id='main-category-filter',
                    options=[{'label': 'Все категории', 'value': 'all'}] + 
                            [{'label': i, 'value': i} for i in df['question_category'].unique()],
                    value=['all'],
                    multi=True,
                    placeholder='Выберите категорию(и)'
                )
            ], width=4)
        ], style={'padding': '20px', 'margin-bottom': '0px'})
    ], style={'display': 'none'}),
    
    # Фильтры для вкладки "Работа модели"
    html.Div(id='metrics-filters-container', children=[
        dbc.Row([
            dbc.Col([
                dcc.DatePickerRange(
                    id='metrics-date-filter',
                    min_date_allowed=metrics_df['date'].min(),
                    max_date_allowed=metrics_df['date'].max(),
                    start_date=metrics_df['date'].min(),
                    end_date=metrics_df['date'].max(),
                    display_format='YYYY-MM-DD',
                    style={'width': '100%'}
                )
            ], width=12)
        ], style={'padding': '20px', 'margin-bottom': '0px'})
    ], style={'display': 'none'}),
    
    # Контейнер для содержимого вкладок
    html.Div(id='tabs-content', style={'padding': '20px'}),
    
    # Скрытые компоненты для скачивания
    dcc.Download(id="download-full-csv"),
    dcc.Download(id="download-full-excel"),
    dcc.Download(id="download-full-json"),
    dcc.Download(id="download-metrics-csv"),
    dcc.Download(id="download-metrics-excel"),
    dcc.Download(id="download-metrics-json")
])

# Колбэк для управления видимостью фильтров
@app.callback(
    [Output('main-filters-container', 'style'),
     Output('metrics-filters-container', 'style')],
    Input('main-tabs', 'value')
)
def toggle_filters_visibility(tab):
    if tab == 'tab-1':
        return {'display': 'block'}, {'display': 'none'}
    elif tab == 'tab-2':
        return {'display': 'block'}, {'display': 'none'}
    elif tab == 'tab-3':
        return {'display': 'none'}, {'display': 'block'}
    else:
        return {'display': 'none'}, {'display': 'none'}

# Колбэк для обновления доступных категорий
@app.callback(
    [Output('main-category-filter', 'options'),
     Output('main-category-filter', 'value')],
    [Input('main-campus-filter', 'value'),
     Input('main-education-filter', 'value')],
    [State('main-category-filter', 'value')]
)
def update_category_options(selected_campuses, selected_educations, current_categories):
    filtered_df = df.copy()
    
    if 'all' not in selected_campuses:
        filtered_df = filtered_df[filtered_df['campus'].isin(selected_campuses)]
    
    if 'all' not in selected_educations:
        filtered_df = filtered_df[filtered_df['education_level'].isin(selected_educations)]
    
    available_categories = filtered_df['question_category'].unique()
    
    options = [{'label': 'Все категории', 'value': 'all'}] + \
              [{'label': cat, 'value': cat} for cat in available_categories]
    
    if current_categories:
        if 'all' in current_categories and len(current_categories) > 1:
            current_categories = [cat for cat in current_categories if cat != 'all']
        current_categories = [cat for cat in current_categories if cat == 'all' or cat in available_categories]
        if not current_categories and available_categories.size > 0:
            current_categories = ['all']
    else:
        current_categories = ['all']
    
    return options, current_categories

# Колбэк для обновления контента
@app.callback(
    Output('tabs-content', 'children'),
    [Input('main-tabs', 'value'),
     Input('main-campus-filter', 'value'),
     Input('main-education-filter', 'value'),
     Input('main-category-filter', 'value'),
     Input('metrics-date-filter', 'start_date'),
     Input('metrics-date-filter', 'end_date')]
)
def update_content(tab, selected_campuses, selected_educations, selected_categories, start_date, end_date):
    if tab == 'tab-1':
        filtered_df = df.copy()
        
        if 'all' not in selected_campuses:
            filtered_df = filtered_df[filtered_df['campus'].isin(selected_campuses)]
        
        if 'all' not in selected_educations:
            filtered_df = filtered_df[filtered_df['education_level'].isin(selected_educations)]
        
        if 'all' not in selected_categories:
            filtered_df = filtered_df[filtered_df['question_category'].isin(selected_categories)]
        
        total_users = len(filtered_df)
        satisfied_users = len(filtered_df[filtered_df.refined_question.isna()])
        satisfaction_rate = round((satisfied_users / total_users) * 100, 2) if total_users > 0 else 0

        avg_time = round(filtered_df.response_time.mean(), 2) if filtered_df.response_time.mean() > 0 else 0
        
        metrics_row = dbc.Row([
            dbc.Col([
                html.Div([
                    html.H6('Процент удовлетворенных пользователей'),
                    html.H1(f'{satisfaction_rate}%', style={'color': text_color})
                ], className='metric-card', style={
                    'padding': '20px', 
                    'border-radius': '10px', 
                    'box-shadow': '0 4px 6px rgba(0,0,0,0.1)',
                    'background-color': 'white'
                })
            ], width=4),
            
            dbc.Col([
                html.Div([
                    html.H6('Оценка производительности чат-бота'),
                    html.H1('59.63%', style={'color': text_color})
                ], className='metric-card', style={
                    'padding': '20px', 
                    'border-radius': '10px', 
                    'box-shadow': '0 4px 6px rgba(0,0,0,0.1)',
                    'background-color': 'white'
                })
            ], width=4),
            
            dbc.Col([
                html.Div([
                    html.H6('Среднее время ответа, сек'),
                    html.H1(f'{avg_time}', style={'color': text_color})
                ], className='metric-card', style={
                    'padding': '20px', 
                    'border-radius': '10px', 
                    'box-shadow': '0 4px 6px rgba(0,0,0,0.1)',
                    'background-color': 'white'
                })
            ], width=4)
        ], className='mb-4')
        
        if len(filtered_df) > 0:
            campus_fig = create_campus_plot(filtered_df)
            education_fig = create_education_plot(filtered_df)
            categories_fig = create_categories_plot(filtered_df)
            
            graphs_row1 = dbc.Row([
                dbc.Col(dcc.Graph(
                    figure=campus_fig,
                    config={'displayModeBar': True, 'displaylogo': False, 'modeBarButtons': [['toImage']]}
                ), width=6, style={'padding': '10px'}),
                dbc.Col(dcc.Graph(
                    figure=education_fig,
                    config={'displayModeBar': True, 'displaylogo': False, 'modeBarButtons': [['toImage']]}
                ), width=6, style={'padding': '10px'})
            ], className='mb-4')
            
            graphs_row2 = dbc.Row([
                dbc.Col(dcc.Graph(
                    figure=categories_fig,
                    config={'displayModeBar': True, 'displaylogo': False, 'modeBarButtons': [['toImage']]}
                ), width=12, style={'padding': '10px'})
            ])
            
            return html.Div([metrics_row, graphs_row1, graphs_row2])
        else:
            return html.Div([metrics_row, html.P("Нет данных для отображения")])
    
    elif tab == 'tab-2':
        filtered_df = df.copy()
        
        if 'all' not in selected_campuses:
            filtered_df = filtered_df[filtered_df['campus'].isin(selected_campuses)]
        
        if 'all' not in selected_educations:
            filtered_df = filtered_df[filtered_df['education_level'].isin(selected_educations)]
        
        if 'all' not in selected_categories:
            filtered_df = filtered_df[filtered_df['question_category'].isin(selected_categories)]
        
        if len(filtered_df) > 0:
            columns_to_show = ['campus', 'education_level', 'question_category', 
                             'user_question', 'refined_question']
            filtered_df = filtered_df[columns_to_show]
            
            return html.Div([
                dash_table.DataTable(
                    data=filtered_df.to_dict('records'),
                    columns=[{'name': col, 'id': col} for col in columns_to_show],
                    page_size=10,
                    style_table={
                        'height': '400px',
                        'overflowY': 'auto',
                        'width': '100%'
                    },
                    style_cell={
                        'textAlign': 'left',
                        'padding': '10px',
                        'whiteSpace': 'normal',
                        'height': 'auto',
                        'maxWidth': '300px'
                    },
                    style_header={
                        'backgroundColor': 'rgb(230, 230, 230)',
                        'fontWeight': 'bold',
                        'padding': '10px'
                    }
                )
            ], style={'width': '100%'})
        else:
            return html.Div("Нет данных для отображения")
    
    elif tab == 'tab-3':
        filtered_metrics = metrics_df[
            (metrics_df['date'] >= start_date) & 
            (metrics_df['date'] <= end_date)
        ]
        
        metrics = {
            'mean_answer_correctness_literal': filtered_metrics['mean_answer_correctness_literal'].mean(),
            'mean_answer_correctness_neural': filtered_metrics['mean_answer_correctness_neural'].mean(),
            'mean_context_precision': filtered_metrics['mean_context_precision'].mean(),
            'mean_context_recall': filtered_metrics['mean_context_recall'].mean(),
            'mean_response_time': df['response_time'].mean()
        }
        
        metrics_row = dbc.Row([
            dbc.Col([
                html.Div([
                    html.H6('Корректность ответов (literal)'),
                    html.H1(f"{metrics['mean_answer_correctness_literal']:.2%}", 
                           style={'color': text_color})
                ], style={
                    'padding': '20px', 
                    'border-radius': '10px', 
                    'box-shadow': '0 4px 6px rgba(0,0,0,0.1)',
                    'background-color': 'white'
                })
            ], width=3),
            
            dbc.Col([
                html.Div([
                    html.H6('Корректность ответов (neural)'),
                    html.H1(f"{metrics['mean_answer_correctness_neural']:.2%}", 
                           style={'color': text_color})
                ], style={
                    'padding': '20px', 
                    'border-radius': '10px', 
                    'box-shadow': '0 4px 6px rgba(0,0,0,0.1)',
                    'background-color': 'white'
                })
            ], width=3),
            
            dbc.Col([
                html.Div([
                    html.H6('Точность контекста'),
                    html.H1(f"{metrics['mean_context_precision']:.2%}", 
                           style={'color': text_color})
                ], style={
                    'padding': '20px', 
                    'border-radius': '10px', 
                    'box-shadow': '0 4px 6px rgba(0,0,0,0.1)',
                    'background-color': 'white'
                })
            ], width=3),
            
            dbc.Col([
                html.Div([
                    html.H6('Полнота контекста'),
                    html.H1(f"{metrics['mean_context_recall']:.2%}", 
                           style={'color': text_color})
                ], style={
                    'padding': '20px', 
                    'border-radius': '10px', 
                    'box-shadow': '0 4px 6px rgba(0,0,0,0.1)',
                    'background-color': 'white'
                })
            ], width=3),
        ], className='mb-4')
        
        figs = []
        metrics_to_plot = [
            'mean_answer_correctness_literal',
            'mean_answer_correctness_neural',
            'mean_context_precision',
            'mean_context_recall'
        ]
        
        for metric in metrics_to_plot:
            fig = px.line(
                filtered_metrics,
                x='date',
                y=metric,
                title=metric.replace('_', ' ').title(),
                color_discrete_sequence=coolors,
                markers=True
            )
            fig.update_layout(
                plot_bgcolor='white',
                paper_bgcolor='white',
                margin=dict(l=20, r=20, t=40, b=20)
            )
            figs.append(fig)
        
        graphs_row1 = dbc.Row([
            dbc.Col(dcc.Graph(
                figure=figs[0],
                config={'displayModeBar': True, 'displaylogo': False, 'modeBarButtons': [['toImage']]}
            ), width=6, style={'padding': '10px'}),
            dbc.Col(dcc.Graph(
                figure=figs[1],
                config={'displayModeBar': True, 'displaylogo': False, 'modeBarButtons': [['toImage']]}
            ), width=6, style={'padding': '10px'})
        ], className='mb-4')
        
        graphs_row2 = dbc.Row([
            dbc.Col(dcc.Graph(
                figure=figs[2],
                config={'displayModeBar': True, 'displaylogo': False, 'modeBarButtons': [['toImage']]}
            ), width=6, style={'padding': '10px'}),
            dbc.Col(dcc.Graph(
                figure=figs[3],
                config={'displayModeBar': True, 'displaylogo': False, 'modeBarButtons': [['toImage']]}
            ), width=6, style={'padding': '10px'})
        ])
        
        return html.Div([metrics_row, graphs_row1, graphs_row2])
    
    elif tab == 'tab-4':
        return html.Div([
            dbc.Container([
                dbc.Row([
                    dbc.Col([
                        html.H4("Скачать полные данные", className="mb-3"),
                        dbc.ButtonGroup([
                            dbc.Button("CSV", id="btn-download-full-csv", color="primary", className="me-2"),
                            dbc.Button("Excel", id="btn-download-full-excel", color="primary", className="me-2"),
                            dbc.Button("JSON", id="btn-download-full-json", color="primary"),
                        ], className="mb-4"),
                        
                        html.H4("Скачать метрики", className="mt-4 mb-3"),
                        dbc.ButtonGroup([
                            dbc.Button("CSV", id="btn-download-metrics-csv", color="primary", className="me-2"),
                            dbc.Button("Excel", id="btn-download-metrics-excel", color="primary", className="me-2"),
                            dbc.Button("JSON", id="btn-download-metrics-json", color="primary"),
                        ]),
                    ], width=12),
                ]),
            ], className="mt-4", style={'padding': '20px'})
        ])

# Функции для создания графиков
def create_campus_plot(filtered_df):
    temp = filtered_df.campus.value_counts().reset_index()
    fig = px.pie(
        temp,
        names='campus',
        values='count',
        title='Кампусы',
        labels={'campus': 'Кампус', 'count': 'Количество'},
        color_discrete_sequence=coolors,
        hover_name='campus',
        hole=0.65
    )
    fig.update_traces(hovertemplate='<b>%{label}</b><br>Количество: %{value}<extra></extra>')
    return fig

def create_education_plot(filtered_df):
    temp = filtered_df.education_level.value_counts().reset_index()
    fig = px.pie(
        temp,
        names='education_level',
        values='count',
        labels={'education_level': 'Уровень образования', 'count': 'Количество'},
        color_discrete_sequence=coolors,
        title='Образование',
        hover_name='education_level',
        hole=0.65
    )
    fig.update_traces(hovertemplate='<b>%{label}</b><br>Количество: %{value}<extra></extra>')
    return fig

def create_categories_plot(filtered_df):
    temp = filtered_df.question_category.value_counts().reset_index().sort_values(by='count', ascending=True)
    fig = px.bar(
        temp,
        y='question_category',
        x='count',
        title='Категории вопросов',
        color_discrete_sequence=coolors,
        labels={'question_category': '', 'count': 'Количество'},
        hover_name='question_category'
    )
    fig.update_layout(
        plot_bgcolor='white',
        paper_bgcolor='white',
        title_x=0,
        title_y=0.99,
        margin=dict(l=0, r=0, t=30, b=0),
        yaxis=dict(showgrid=True, gridcolor='lightgray'),
        xaxis=dict(showgrid=True, gridcolor='lightgray'),
    )
    fig.update_traces(width=0.7, hovertemplate='<b>%{label}</b><br>Количество: %{value}<extra></extra>')
    return fig

# Колбэки для скачивания данных
@app.callback(
    Output("download-full-csv", "data"),
    Input("btn-download-full-csv", "n_clicks"),
    prevent_initial_call=True
)
def download_full_csv(n_clicks):
    if n_clicks is None:
        return dash.no_update
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    return dcc.send_data_frame(df.to_csv, f"full_data_{timestamp}.csv", index=False)

@app.callback(
    Output("download-full-excel", "data"),
    Input("btn-download-full-excel", "n_clicks"),
    prevent_initial_call=True
)
def download_full_excel(n_clicks):
    if n_clicks is None:
        return dash.no_update
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    return dcc.send_data_frame(df.to_excel, f"full_data_{timestamp}.xlsx", index=False)

@app.callback(
    Output("download-full-json", "data"),
    Input("btn-download-full-json", "n_clicks"),
    prevent_initial_call=True
)
def download_full_json(n_clicks):
    if n_clicks is None:
        return dash.no_update
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    return dict(content=df.to_json(orient="records"), filename=f"full_data_{timestamp}.json")

@app.callback(
    Output("download-metrics-csv", "data"),
    Input("btn-download-metrics-csv", "n_clicks"),
    prevent_initial_call=True
)
def download_metrics_csv(n_clicks):
    if n_clicks is None:
        return dash.no_update
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    return dcc.send_data_frame(metrics_df.to_csv, f"metrics_{timestamp}.csv", index=False)

@app.callback(
    Output("download-metrics-excel", "data"),
    Input("btn-download-metrics-excel", "n_clicks"),
    prevent_initial_call=True
)
def download_metrics_excel(n_clicks):
    if n_clicks is None:
        return dash.no_update
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    return dcc.send_data_frame(metrics_df.to_excel, f"metrics_{timestamp}.xlsx", index=False)

@app.callback(
    Output("download-metrics-json", "data"),
    Input("btn-download-metrics-json", "n_clicks"),
    prevent_initial_call=True
)
def download_metrics_json(n_clicks):
    if n_clicks is None:
        return dash.no_update
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    return dict(content=metrics_df.to_json(orient="records"), filename=f"metrics_{timestamp}.json")

if __name__ == '__main__':
    app.run_server(debug=False)

C:\Users\gnrly\AppData\Local\Temp\ipykernel_17200\249260398.py:26: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

